In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
TEAMS_CSV = Path("../data/raw/Teams.csv")

In [3]:
teams_df = pd.read_csv(TEAMS_CSV)
teams_df.head()

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,1884,UA,ALT,ALT,NaN,10,25,NaN,6,19,...,4,0.862,Altoona Mountain City,NaN,NaN,101,109,ALT,ALT,ALT
1,1961,AL,LAA,ANA,NaN,8,162,82.0,70,91,...,154,0.969,Los Angeles Angels,Wrigley Field (LA),603510.0,111,112,LAA,LAA,LAA
2,1962,AL,LAA,ANA,NaN,3,162,81.0,86,76,...,153,0.972,Los Angeles Angels,Dodger Stadium,1144063.0,97,97,LAA,LAA,LAA
3,1963,AL,LAA,ANA,NaN,9,161,81.0,70,91,...,155,0.974,Los Angeles Angels,Dodger Stadium,821015.0,94,94,LAA,LAA,LAA
4,1964,AL,LAA,ANA,NaN,5,162,81.0,82,80,...,168,0.978,Los Angeles Angels,Dodger Stadium,760439.0,90,90,LAA,LAA,LAA


In [4]:
teams_df.columns

Index(['yearID', 'lgID', 'teamID', 'franchID', 'divID', 'Rank', 'G', 'Ghome',
       'W', 'L', 'DivWin', 'WCWin', 'LgWin', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'CS', 'HBP', 'SF', 'RA', 'ER', 'ERA',
       'CG', 'SHO', 'SV', 'IPouts', 'HA', 'HRA', 'BBA', 'SOA', 'E', 'DP', 'FP',
       'name', 'park', 'attendance', 'BPF', 'PPF', 'teamIDBR',
       'teamIDlahman45', 'teamIDretro'],
      dtype='object')

In [5]:
shape = teams_df.shape
num_cols = teams_df.shape[1]
num_rows = teams_df.shape[0]

print(f"Número de columnas: {num_cols}")
print(f"Número de filas: {num_rows}")

Número de columnas: 48
Número de filas: 3075


In [6]:
mask = teams_df["lgID"].isin(["AL", "NL"]) & teams_df["yearID"].between(2000, 2019)
df = teams_df.loc[
    mask,
    [
        "yearID",
        "lgID",
        "teamID",
        "franchID",
        "name",
        "G",
        "W",
        "L",
        "R",
        "RA",
        "ERA",
        "HR",
    ],
].copy()
df.head()

,yearID,lgID,teamID,franchID,name,G,W,L,R,RA,ERA,HR
40,2000,AL,ANA,ANA,Anaheim Angels,162,82,80,864,869,5.00,236
41,2001,AL,ANA,ANA,Anaheim Angels,162,75,87,691,730,4.20,158
42,2002,AL,ANA,ANA,Anaheim Angels,162,99,63,851,644,3.69,152
43,2003,AL,ANA,ANA,Anaheim Angels,162,77,85,736,743,4.28,150
44,2004,AL,ANA,ANA,Anaheim Angels,162,92,70,836,734,4.28,162


In [7]:
df["RunDiff"] = df["R"] - df["RA"]
df["logHR1"] = np.log(df["HR"] + 1)
df["logERA"] = np.log(df["ERA"] + 1)
df["season_date"] = pd.to_datetime(df["yearID"].astype(str) + "-07-01")
df["team_year"] = df["teamID"] + "_" + df["yearID"].astype(str)
df.head()

,yearID,lgID,teamID,franchID,name,G,W,L,R,RA,ERA,HR,RunDiff,logHR1,logERA,season_date,team_year
40,2000,AL,ANA,ANA,Anaheim Angels,162,82,80,864,869,5.00,236,-5,5.468060,1.791759,2000-07-01,ANA_2000
41,2001,AL,ANA,ANA,Anaheim Angels,162,75,87,691,730,4.20,158,-39,5.068904,1.648659,2001-07-01,ANA_2001
42,2002,AL,ANA,ANA,Anaheim Angels,162,99,63,851,644,3.69,152,207,5.030438,1.545433,2002-07-01,ANA_2002
43,2003,AL,ANA,ANA,Anaheim Angels,162,77,85,736,743,4.28,150,-7,5.017280,1.663926,2003-07-01,ANA_2003
44,2004,AL,ANA,ANA,Anaheim Angels,162,92,70,836,734,4.28,162,102,5.093750,1.663926,2004-07-01,ANA_2004


In [8]:
print("Duplicados: ", df[["yearID", "teamID"]].duplicated().sum())
print("Equipos por año:\n", df["yearID"].value_counts().sort_index())

Duplicados:  0
Equipos por año:
 yearID
2000    30
2001    30
2002    30
2003    30
2004    30
2005    30
2006    30
2007    30
2008    30
2009    30
2010    30
2011    30
2012    30
2013    30
2014    30
2015    30
2016    30
2017    30
2018    30
2019    30
Name: count, dtype: int64


In [9]:
OUT = Path("../data/processed/master_teams_2000_2019.csv")
OUT.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUT, index=False)
print(f"Archivo guardado en {OUT.resolve()}")

Archivo guardado en /Users/enriquegomeztagle/Desktop/0241823/UP/MCD-ProyectoFinalEconometria-DeterminantesVictoriasMLB/data/processed/master_teams_2000_2019.csv


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, 40 to 3068
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   yearID       600 non-null    int64         
 1   lgID         600 non-null    object        
 2   teamID       600 non-null    object        
 3   franchID     600 non-null    object        
 4   name         600 non-null    object        
 5   G            600 non-null    int64         
 6   W            600 non-null    int64         
 7   L            600 non-null    int64         
 8   R            600 non-null    int64         
 9   RA           600 non-null    int64         
 10  ERA          600 non-null    float64       
 11  HR           600 non-null    int64         
 12  RunDiff      600 non-null    int64         
 13  logHR1       600 non-null    float64       
 14  logERA       600 non-null    float64       
 15  season_date  600 non-null    datetime64[ns]
 16  team_year  